In [2]:
import pandas as pd

Filter uniprot by (protein_name:"Tryptophan synthase beta chain") - should yield 55,634 results and download as TSV

In [ ]:
!curl "https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Cid%2Cprotein_name%2Csequence&format=tsv&query=%28%28protein_name%3A%22Tryptophan+synthase+beta+chain%22%29%29" > raw_data/trpB.tsv.gz
!gunzip raw_data/trpB.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7515k    0 7515k    0     0   387k      0 --:--:--  0:00:19 --:--:--  484k


In [26]:
df = pd.read_csv("raw_data/trpB.tsv", sep="\t")
df

,Entry,Entry Name,Protein names,Sequence
0,P0A2K1,TRPB_SALTY,Tryptophan synthase beta chain (EC 4.2.1.20),MTTLLNPYFGEFGGMYVPQILMPALNQLEEAFVSAQKDPEFQAQFA...
1,P0A879,TRPB_ECOLI,Tryptophan synthase beta chain (EC 4.2.1.20),MTTLLNPYFGEFGGMYVPQILMPALRQLEEAFVSAQKDPEFQAQFN...
2,P14671,TRPB1_ARATH,"Tryptophan synthase beta chain 1, chloroplasti...",MAASGTSATFRASVSSAPSSSSQLTHLKSPFKAVKYTPLPSSRSKS...
3,A0A086ZWJ4,A0A086ZWJ4_9BIFI,Multifunctional fusion protein [Includes: Indo...,MSVLDELVAGALEDQRTRERSVSLEDVKRAALAAPAPIDAMRWLKR...
4,A0A0T5ZJQ8,A0A0T5ZJQ8_9CHLR,Multifunctional fusion protein [Includes: N-(5...,MTAAVLDEIAARRRADVTAELGDRALTELHRAAERAPAPRDAVGPL...
...,...,...,...,...
55629,X6HIN1,X6HIN1_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MDAMCNCRPFVGYSDAYALPRIIKQENLYLAQFAFMKLLPAKYIIE...
55630,X6K746,X6K746_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MENAKREHCNSGLAFGGNKLRKLEYIIPDAIASNADTLVTIGGVQS...
55631,X6PCE9,X6PCE9_RETFI,Tryptophan synthase beta chain-like PALP domai...,MQLGWQHQCKIVVCQTEHGQTLASAVQNAKEGEKTTPVRTHVIDPC...
55632,X7RXU0,X7RXU0_FUSNU,Tryptophan synthase beta chain-like PALP domai...,MQQEKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...


In [4]:
#save sequences to fasta file
with open("raw_data/trpB.fasta", "w") as f:
    for i, row in df.iterrows():
        f.write(f">{row['Entry']}\n{row['Sequence'] }\n")

In [11]:
#run mmseqs clustering
!mmseqs easy-cluster raw_data/trpB.fasta raw_data/trpB_res_70 raw_data/tmp --min-seq-id 0.7 #-c 0.8 --cov-mode 1

easy-cluster raw_data/trpB.fasta raw_data/trpB_res_70 raw_data/tmp --min-seq-id 0.7 

MMseqs Version:                     	16.747c6
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	0
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask residu

In [12]:
!mmseqs easy-cluster raw_data/trpB.fasta raw_data/trpB_res_90 raw_data/tmp --min-seq-id 0.9 #-c 0.8 --cov-mode 1

easy-cluster raw_data/trpB.fasta raw_data/trpB_res_90 raw_data/tmp --min-seq-id 0.9 

MMseqs Version:                     	16.747c6
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	0
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask residu

In [27]:
cluster_res = pd.read_csv("raw_data/trpB_res_70_cluster.tsv", sep="\t", header=None)
cluster_res.columns = ["cluster_70", "Entry"]
df = pd.merge(df, cluster_res, on="Entry")

cluster_res = pd.read_csv("raw_data/trpB_res_90_cluster.tsv", sep="\t", header=None)
cluster_res.columns = ["cluster_90", "Entry"]
df = pd.merge(df, cluster_res, on="Entry")
df

,Entry,Entry Name,Protein names,Sequence,cluster_70,cluster_90
0,P0A2K1,TRPB_SALTY,Tryptophan synthase beta chain (EC 4.2.1.20),MTTLLNPYFGEFGGMYVPQILMPALNQLEEAFVSAQKDPEFQAQFA...,A0A097L0A6,A0A447JIV4
1,P0A879,TRPB_ECOLI,Tryptophan synthase beta chain (EC 4.2.1.20),MTTLLNPYFGEFGGMYVPQILMPALRQLEEAFVSAQKDPEFQAQFN...,A0A447JIV4,A0A447JIV4
2,P14671,TRPB1_ARATH,"Tryptophan synthase beta chain 1, chloroplasti...",MAASGTSATFRASVSSAPSSSSQLTHLKSPFKAVKYTPLPSSRSKS...,A0A856MHW4,P14671
3,A0A086ZWJ4,A0A086ZWJ4_9BIFI,Multifunctional fusion protein [Includes: Indo...,MSVLDELVAGALEDQRTRERSVSLEDVKRAALAAPAPIDAMRWLKR...,A0A086ZWJ4,A0A086ZWJ4
4,A0A0T5ZJQ8,A0A0T5ZJQ8_9CHLR,Multifunctional fusion protein [Includes: N-(5...,MTAAVLDEIAARRRADVTAELGDRALTELHRAAERAPAPRDAVGPL...,A0A0T5ZJQ8,A0A0T5ZJQ8
...,...,...,...,...,...,...
55629,X6HIN1,X6HIN1_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MDAMCNCRPFVGYSDAYALPRIIKQENLYLAQFAFMKLLPAKYIIE...,X6HIN1,X6HIN1
55630,X6K746,X6K746_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MENAKREHCNSGLAFGGNKLRKLEYIIPDAIASNADTLVTIGGVQS...,X6K746,X6K746
55631,X6PCE9,X6PCE9_RETFI,Tryptophan synthase beta chain-like PALP domai...,MQLGWQHQCKIVVCQTEHGQTLASAVQNAKEGEKTTPVRTHVIDPC...,X6PCE9,X6PCE9
55632,X7RXU0,X7RXU0_FUSNU,Tryptophan synthase beta chain-like PALP domai...,MQQEKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,U7TP76,U7TP76


In [28]:
df["cluster_70"].value_counts()

cluster_70
A0A447JIV4    2158
A0A212LJQ8    1867
A0A967L7D8    1560
A0A2Z4J558    1089
N2J1A8        1088
              ... 
A0A1G3LQC0       1
A0A1G3IQE2       1
A0A1G2ZYD8       1
A0A1G2SM23       1
A0A1J4S0G0       1
Name: count, Length: 8856, dtype: int64

In [29]:
df["cluster_90"].value_counts()

cluster_90
A0A447JIV4    899
A0A6A9K3S4    528
A0A3R9UBE6    491
A0A2S2FB95    284
A0A918HTY3    252
             ... 
X5ZL31          1
X6BCT3          1
X1VT60          1
X1VTC1          1
X1VTW7          1
Name: count, Length: 25136, dtype: int64

In [30]:
#randomly sample 5% of the clusters at 90% identity and mark them as validation set
import numpy as np
np.random.seed(42)
test_clusters = df["cluster_90"].unique()
test_clusters = np.random.choice(test_clusters, int(len(test_clusters)*0.05), replace=False)
df["Split"] = df["cluster_90"].apply(lambda x: x in test_clusters)
#map True to validation and False to training
df["Split"] = df["Split"].apply(lambda x: "validation" if x else "train")

In [31]:
df

,Entry,Entry Name,Protein names,Sequence,cluster_70,cluster_90,Split
0,P0A2K1,TRPB_SALTY,Tryptophan synthase beta chain (EC 4.2.1.20),MTTLLNPYFGEFGGMYVPQILMPALNQLEEAFVSAQKDPEFQAQFA...,A0A097L0A6,A0A447JIV4,train
1,P0A879,TRPB_ECOLI,Tryptophan synthase beta chain (EC 4.2.1.20),MTTLLNPYFGEFGGMYVPQILMPALRQLEEAFVSAQKDPEFQAQFN...,A0A447JIV4,A0A447JIV4,train
2,P14671,TRPB1_ARATH,"Tryptophan synthase beta chain 1, chloroplasti...",MAASGTSATFRASVSSAPSSSSQLTHLKSPFKAVKYTPLPSSRSKS...,A0A856MHW4,P14671,train
3,A0A086ZWJ4,A0A086ZWJ4_9BIFI,Multifunctional fusion protein [Includes: Indo...,MSVLDELVAGALEDQRTRERSVSLEDVKRAALAAPAPIDAMRWLKR...,A0A086ZWJ4,A0A086ZWJ4,train
4,A0A0T5ZJQ8,A0A0T5ZJQ8_9CHLR,Multifunctional fusion protein [Includes: N-(5...,MTAAVLDEIAARRRADVTAELGDRALTELHRAAERAPAPRDAVGPL...,A0A0T5ZJQ8,A0A0T5ZJQ8,train
...,...,...,...,...,...,...,...
55629,X6HIN1,X6HIN1_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MDAMCNCRPFVGYSDAYALPRIIKQENLYLAQFAFMKLLPAKYIIE...,X6HIN1,X6HIN1,train
55630,X6K746,X6K746_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MENAKREHCNSGLAFGGNKLRKLEYIIPDAIASNADTLVTIGGVQS...,X6K746,X6K746,train
55631,X6PCE9,X6PCE9_RETFI,Tryptophan synthase beta chain-like PALP domai...,MQLGWQHQCKIVVCQTEHGQTLASAVQNAKEGEKTTPVRTHVIDPC...,X6PCE9,X6PCE9,train
55632,X7RXU0,X7RXU0_FUSNU,Tryptophan synthase beta chain-like PALP domai...,MQQEKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,U7TP76,U7TP76,train


In [32]:
df[df["Split"] == "validation"]

,Entry,Entry Name,Protein names,Sequence,cluster_70,cluster_90,Split
41,A0A063ZPA4,A0A063ZPA4_9EURY,Tryptophan synthase beta chain (EC 4.2.1.20),MSETERGREGERTDRTTADGQFGRYGGQYVPEALMPALEELEDAYE...,A0A482XUD6,A0A063ZPA4,validation
42,A0A066TVU0,A0A066TVU0_9PSEU,Tryptophan synthase beta chain (EC 4.2.1.20),MAEQTKYVLDEADLPTQWYNVIPDLPEPPPPPLHPGTREPVGPDDL...,A0A3D9V4J6,M2QF06,validation
45,A0A075V324,A0A075V324_9PSEU,Tryptophan synthase beta chain (EC 4.2.1.20),MAERTKYILDEADMPTRWYNVVPDLPEPPPPPLHPGTREPVGPDDL...,A0A3D9V4J6,M2QF06,validation
63,A0A086ZRR8,A0A086ZRR8_9BIFI,Multifunctional fusion protein [Includes: Indo...,MSVLDELVAGATQDMRQRAETVTLDDLKRLAGTVPPPLDAKQWLKR...,A0A6L4X374,A0A086ZRR8,validation
96,A0A098S1U5,A0A098S1U5_9BACT,Tryptophan synthase beta chain (EC 4.2.1.20),MNTRKILLSENEIPEAWYNIVHDMPNKPLPPLHPGTKEPIGPDALA...,A0A1Q3K6R5,A0A5C6S4I3,validation
...,...,...,...,...,...,...,...
55547,X1S366,X1S366_9ZZZZ,Tryptophan synthase beta chain-like PALP domai...,PIKKLDVIGEKKGSNIWVKLENLNPSGSLKDRIALQIIEDAEKNGD...,X1S366,X1S366,validation
55559,X1TGE0,X1TGE0_9ZZZZ,Tryptophan synthase beta chain-like PALP domai...,GTITGVGEVLKQKKPSVKIIAVEPADSPVLSGGSPGSHKIQGIGAG...,X1EKI6,X1TGE0,validation
55572,X1UGG4,X1UGG4_9ZZZZ,Tryptophan synthase beta chain-like PALP domai...,SGNTGIALAWLGRQKGYKVTIVMPDNMSMERRQLLKINGAELILTN...,X1UGG4,X1UGG4,validation
55617,X6BG06,X6BG06_9HYPH,Tryptophan synthase beta chain-like PALP domai...,MLAARKLLPFSGYSDAYALPSIIRCETNLYLAQFAFMKLLPAKYII...,X6HIN1,X6BG06,validation


In [33]:
df.to_csv("trpB_processed.csv", index=False)